In [2]:
class Node:
    
    def __init__(self, data = None):
        self._data = data
        self._next = None
        
    def getData(self):
        return self._data
        
    def getNext(self):
        return self._next
    
    def setNext(self, next_):
        self._next = next_

In [3]:
class LinkedList:
    
    def __init__(self):
        self._head = None
        
    def getHead(self):
        return self._head
        
    def add(self, data):
        if self._head == None:
            self._head = Node(data)
        else:
            current = self._head
            while current.getNext() != None:
                current = current.getNext()
            current.setNext(Node(data))
    
    def remove(self, data):
        if self._head == None:
            return None
        if self._head.getData() == data:
            self._head = self._head.getNext()
            return data
        current = self._head
        while current.getNext().getData() != data and current.getNext().getNext() != None:
            current = current.getNext()
        if current.getNext().getData() == data:
            current.setNext(current.getNext().getNext())
            return data

In [4]:
class HashMap:
    
    def __init__(self, length):
        self._length = length
        self._array = [LinkedList() for _ in range(length)]
        
    def getLength(self):
        return self._length
        
    def add(self, data, index):
        if index >= self._length:
            self.extend(index)
        self._array[index].add(data)
            
    def extend(self, length):
        if length < self._length:
            return
        self._array += [LinkedList() for _ in range(self._length, length + 1)]
        self._length = length + 1
        
    def iterate(self, index):
        current = self._array[index].getHead()
        while current != None:
            yield current.getData()
            current = current.getNext()
            
    def remove(self, data, index):
        self._array[index].remove(data)
        
    def print_(self):
        for i in range(self._length):
            print('head -> ' + ' -> '.join([str(node.getId()) for node in self.iterate(i)]))

In [5]:
class Brick:
    
    def __init__(self, line):
        self._name = line
        self._ends = [[int(char) for char in piece.split(',')] for piece in line.split('~')]
    
    def getName(self):
        return self._name
    
    def getEnd(self, i):
        return self._ends[i]
        
    def getMinHeight(self):
        z0 = self._ends[0][2]
        z1 = self._ends[1][2]
        return min(z0, z1)
    
    def getMaxHeight(self):
        z0 = self._ends[0][2]
        z1 = self._ends[1][2]
        return max(z0, z1)
                    
    def iterateCoordinate(self, i, direction = 1):
        if direction >= 0:
            start = self._ends[0][i]
            end = self._ends[1][i]
        else:
            start = self._ends[1][i]
            end = self._ends[0][i]
        step = 1 if start <= end else -1
        for i in range(start, end + step, step):
            yield i
            
    def lower(self):
        self._ends[0][2] -= 1
        self._ends[1][2] -= 1
        
    def sitsOn(self, brick):
        dic = {}
        for x in self.iterateCoordinate(0):
            for y in self.iterateCoordinate(1):
                for z in self.iterateCoordinate(2):
                    dic[(x, y, z)] = ''
        for x in brick.iterateCoordinate(0):
            for y in brick.iterateCoordinate(1):
                for z in brick.iterateCoordinate(2):
                    if (x, y, z + 1) in dic:
                        return True
        return False

In [6]:
class BrickSnapshot:
    
    def __init__(self, text):
        self._xBound = 10
        self._yBound = 10
        self._zBound = 250
        self._brickArray = self.initBrickArray()
        self._brickIndex = HashMap(self._zBound)
        for line in text.split('\n'):
            brick = Brick(line)
            self.updateMaximumHeight(brick.getMaxHeight())
            self.updateBrickArray(brick, 1)
            self._brickIndex.add(brick, brick.getMinHeight())
    
    def initBrickArray(self):
        return [[[0 for z in range(self._zBound)] \
                 for y in range(self._yBound)] \
                for x in range(self._xBound)]
        
    def updateMaximumHeight(self, height):
        if height >= self._zBound:
            self.extendMaximumHeight(height)
            
    def extendMaximumHeight(self, height):
        if self._zBound > height:
            return
        for x in range(self._xBound):
            for y in range(self._yBound):
                self._brickArray[x][y] += [0 for _ in range(self._zBound, height + 1)]
        self._zBound = height + 1
        
    def updateBrickArray(self, brick, value):
        for x in brick.iterateCoordinate(0):
            for y in brick.iterateCoordinate(1):
                for z in brick.iterateCoordinate(2):
                    self._brickArray[x][y][z] = value
                    
    def getHeight(self):
        return self._brickArray.getLength()
    
    def lowerBricks(self):
        for height in range(1, self._brickIndex.getLength()):
            self.lowerBricksAtHeight(height)
            
    def lowerBricksAtHeight(self, height):
        # track bricks which have been lowered
        lowered = []
        for brick in self._brickIndex.iterate(height):
            if self.canLower(brick):
                self.lowerBrick(brick)
                lowered.append(brick)
            else:
                continue
            while self.canLower(brick):
                self.lowerBrick(brick)
        for brick in lowered:
            self._brickIndex.remove(brick, height)
            self._brickIndex.add(brick, brick.getMinHeight())
    
    def canLower(self, brick):
        minHeight = brick.getMinHeight()
        if minHeight == 1:
            return False
        for x in brick.iterateCoordinate(0):
            for y in brick.iterateCoordinate(1):
                if self._brickArray[x][y][minHeight - 1] == 1:
                    return False
        return True
                
    def lowerBrick(self, brick):
        self.updateBrickArray(brick, 0)
        brick.lower()
        self.updateBrickArray(brick, 1)
    
    def iterate(self):
        for height in range(1, self._brickIndex.getLength()):
            for brick in self._brickIndex.iterate(height):
                yield brick
    
    def print_(self):
        for z in range(1, self._zBound):
            print(z)
            psum = 0
            for y in range(self._yBound):
                arr = [self._brickArray[x][y][z] for x in range(self._xBound)]
                print('\t', ''.join([str(x) for x in arr]))
                psum += sum(arr)
            if psum == 0:
                break
                
    def iterateSupported(self, brick):
        for other in self._brickIndex.iterate(brick.getMaxHeight() + 1):
            if other.sitsOn(brick):
                yield other

In [7]:
class GraphNode(Node):
    
    def __init__(self, data):
        self._data = data
        self._parents = []
        self._children = []
        
    def getData(self):
        return self._data
    
    def getChildren(self):
        return self._children
    
    def addChild(self, child):
        self._children.append(child)
        child._parents.append(self)
        
    def getParents(self):
        return self._parents

In [8]:
class BrickSupportGraph:
    
    def __init__(self, text):
        brickSnapshot = BrickSnapshot(text)
        brickSnapshot.lowerBricks()
        self._nodeIndex = {}
        for brick in brickSnapshot.iterate():
            self._nodeIndex[brick.getName()] = self.makeNode(brick.getName())
        for brick in brickSnapshot.iterate():
            parent = self._nodeIndex[brick.getName()]
            for other in brickSnapshot.iterateSupported(brick):
                node = self._nodeIndex[other.getName()]
                parent.addChild(node)
    
    def getIndex(self):
        return self._nodeIndex
                
    def makeNode(self, brick):
        return GraphNode(brick)
        
    def count(self):
        count = 0
        for key in self._nodeIndex:
            include = True
            node = self._nodeIndex[key]
            children = node.getChildren()
            for child in node.getChildren():
                if len(child.getParents()) == 1:
                    include = False
                    break
            if include:
                count += 1
        return count
        

In [9]:
graph = BrickSupportGraph(text)
graph.count()

416

In [10]:
class ChainReactionBrickSupportGraph(BrickSupportGraph):
    
    #Override
    def count(self):
        count = 0
        index = self.getIndex()
        for key in index:
            # count node if all parents are in the visited set
            node = index[key]
            count += self.countDependentNodes(node)
        return count
    
    def countDependentNodes(self, node):
        count = 0
        visited = {node.getData():''}
        queue = [node]
        while len(queue) > 0:
            curnode = queue.pop(0)
            for child in curnode.getChildren():
                if child.getData() in visited:
                    continue
                keepChild = True
                for parent in child.getParents():
                    if not parent.getData() in visited:
                        keepChild = False
                        break
                if keepChild:
                    count += 1
                    visited[child.getData()] = ''
                    queue.append(child)
        return count
    
        

In [11]:
chainReaction = ChainReactionBrickSupportGraph(text)
chainReaction.count()

60963